In [1]:
import pandas as pd
import numpy as np
import pickle
from collections import OrderedDict
import os
import argparse
import torch
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
import numpy as np
import pickle
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import save_image

In [4]:
%tb
parser = argparse.ArgumentParser()
parser.add_argument('--epochs', type=int, default=100, help='epoch number')
parser.add_argument('--batch_size', type=str, default=39, help='batch size')
parser.add_argument('--lr', type=float, default=1e-6, help='learning rate')
parser.add_argument('--dataset_path', type=str, default='/Users/john/Desktop/106下學期/T_brain競賽/test_100.pickle', help='training dataset path')
parser.add_argument('--gpu_num', type=str, default='0', help='gpu devices number')
opt = parser.parse_args()
print(opt)

cuda = True if torch.cuda.is_available() else False
os.environ["CUDA_VISIBLE_DEVICES"]=opt.gpu_num

# if not os.path.exists('./mlp_img'):
#     os.mkdir('./mlp_img')

# def to_img(x):
#     x = 0.5*(x+1)
#     x = x.clamp(0, 1)
#     x = x.view(x.size(0), 1, 28, 28)
#     return x
size = 100
data = []
# with open('Insured_Amount.pickle', 'rb') as handle:
# with open('new_one.pickle', 'rb') as handle:
with open('test_100.pickle', 'rb') as handle:
    f = pickle.load(handle)
for k,v in f.items():
    data.append(v)
data = np.array(data)
data = data.astype('float32')
for i in range(size):
    for j in range(6):
        for k in range(70):
            if data[i][j][k] > 1:
                data[i][j][k] = np.log(data[i][j][k])
# data = np.load(opt.dataset_path)
data = data.reshape((size, 18*70))
print(data[0])

num_epochs = opt.epochs
batch_size = opt.batch_size
learning_rate = opt.lr

# img_transform = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
# ])

# dataset = MNIST('../data', transform=img_transform)
dataloader = DataLoader(data, batch_size=batch_size, shuffle=True, num_workers=4)
dataloader_all = DataLoader(data, batch_size=size, shuffle=True, num_workers=4)

class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(4*18, 36),
            nn.ReLU(True),
            nn.Linear(36, 18),
            nn.ReLU(True),
            nn.Linear(18, 10),
            # nn.ReLU(True),
            # nn.Linear(12, 3)
        )
        self.decoder = nn.Sequential(
            nn.Linear(10, 18),
            nn.ReLU(True),
            nn.Linear(18, 36),
            nn.ReLU(True),
            nn.Linear(36, 4*18),
            # nn.ReLU(True),
            # nn.Linear(128, 28*28),
            #nn.Tanh()
        )

    def forward(self, x):
        y = self.encoder(x)
        x = self.decoder(y)

        return x, y

class autoencoder_2(nn.Module):
    def __init__(self):
        super(autoencoder_2, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(18*70, 630),
            nn.LeakyReLU(0.1),
            nn.Linear(630, 300),
            nn.LeakyReLU(0.1),
            nn.Linear(300, 150),
            nn.LeakyReLU(0.1),
            nn.Linear(150, 50),
            nn.LeakyReLU(0.1),
            nn.Linear(50, 20),
        )
        self.decoder = nn.Sequential(
            nn.Linear(20, 50),
            nn.LeakyReLU(0.1),
            nn.Linear(50, 150),
            nn.LeakyReLU(0.1),
            nn.Linear(150, 300),
            nn.LeakyReLU(0.1),
            nn.Linear(300, 630),
            nn.LeakyReLU(0.1),
            nn.Linear(630, 18*70)
            #nn.Tanh()
        )

    def forward(self, x):
        y = self.encoder(x)
        x = self.decoder(y)

        return x, y

if __name__ == "__main__":
    model = autoencoder_2()
    # criterion = nn.CrossEntropyLoss()
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,
                                 weight_decay=1e-10)
    out = 0
    output2 = []
    if cuda:
        model.cuda()
        criterion.cuda()
    for epoch in range(num_epochs):
        for data in dataloader:
            # img, _ = data
            # img = img.view(img.size(0), -1)
            # img = Variable(img).cuda()

            d = data.cuda()

            output, encode = model(d)
            loss = criterion(output, d)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print('epoch [{}/{}], loss:{:.4f}'.format(epoch + 1, num_epochs, loss.data[0]))
        # print(encode)
        if (epoch+1) % 10 == 0:
            torch.save(model.state_dict(), './{}epoch model.pth'.format(epoch+1))
        # print(epoch)
        if epoch == num_epochs-1:
            # print('test')
            for d in dataloader_all:
                data = d.cuda()
                out, encode = model(data)
                out = encode.data.cpu().numpy()
                output2.append(out)
                # print(output2)
                # print(len(output2[0]))
                np.save('/home/herokuma/Desktop/encode.npy', out)
                break

    #torch.save(model.state_dict(), './sim_autoencoder.pth')

SystemExit: 2

usage: ipykernel_launcher.py [-h] [--epochs EPOCHS] [--batch_size BATCH_SIZE]
                             [--lr LR] [--dataset_path DATASET_PATH]
                             [--gpu_num GPU_NUM]
ipykernel_launcher.py: error: unrecognized arguments: -f /Users/john/Library/Jupyter/runtime/kernel-6fd35b0e-26d2-482d-862c-61c99190a547.json


SystemExit: 2

/usr/local/anaconda3/envs/drug_design/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2969: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
